In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 121 ms


In [3]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


echodata_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)
labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 15.7 s


In [4]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 4.67 ms


In [5]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df, vaso_dose_72, echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
ventcategory_df, vaso_dose_72, echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 1.06 s


#### all dataframes to melt together
- vaso_dose_72 : 
- ventcategory_df : 
- echodata_72 : 
- labs_all_nosummary_72 : 
- weightfirstday_df : 
- heightfirstday_df : 
- vitals_all_nosummary_72 :
- uti_all_72 :
- bg_all_nosummary_72 :
- rrt_merged_allpt_df :
- gcs72_df : 
- sofa_df_72: 

10/5/18
# Standardizing Columns in df to merge together
### list of columns in master df:
- subject_id
- hadm_id
- icustay_id
- charttime
- label
- value
- t_0
- delta
- days

In [29]:
#vaso_dose_72.head()
vaso_dose_72= vaso_dose_72.rename(
    columns={'starttime':'charttime','label':'vaso_type'}
        ).drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

ValueError: labels ['endtime'] not contained in axis

time: 18.6 ms


In [45]:
list(vaso_dose_72)
vaso_dose_72.head()

#problem, i think it would be best to make a vasotype_vaso_rate, and vasotype_vaso_amount for each vasodil.
#would this b best? need to think of overall structure. 


#goal for 10/5/18: update documentation, committ on github, 
#and look through other data structures and determine the "best" format
#format

,icustay_id,charttime,vaso_rate,vaso_amount,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,0.012030,0.073958,epinephrine,2139-06-02,0 days 16:31:00.000000000
108,200952.0,2139-09-23 15:07:00,0.020004,1.049601,epinephrine,2139-09-23,0 days 15:07:00.000000000
109,200952.0,2139-09-25 12:00:00,0.080068,1.962759,epinephrine,2139-09-23,2 days 12:00:00.000000000
110,200952.0,2139-09-25 21:59:00,0.060601,2.000000,epinephrine,2139-09-23,2 days 21:59:00.000000000
122,200952.0,2139-09-23 23:12:00,0.149992,7.990853,norepinephrine,2139-09-23,0 days 23:12:00.000000000


time: 10.7 ms


In [84]:
test1= vaso_dose_72.set_index(['vaso_type','icustay_id'])
test2= vaso_dose_72.set_index(['vaso_type','vaso_rate','vaso_amount'])
test3= vaso_dose_72.set_index(['vaso_type'])

#del(test)
test1.head(2)
test2.head()
test3.head()

,icustay_id,charttime,vaso_rate,vaso_amount,t_0,delta
vaso_type,,,,,,
epinephrine,200349.0,2139-06-02 16:31:00,0.012030,0.073958,2139-06-02,0 days 16:31:00.000000000
epinephrine,200952.0,2139-09-23 15:07:00,0.020004,1.049601,2139-09-23,0 days 15:07:00.000000000
epinephrine,200952.0,2139-09-25 12:00:00,0.080068,1.962759,2139-09-23,2 days 12:00:00.000000000
epinephrine,200952.0,2139-09-25 21:59:00,0.060601,2.000000,2139-09-23,2 days 21:59:00.000000000
norepinephrine,200952.0,2139-09-23 23:12:00,0.149992,7.990853,2139-09-23,0 days 23:12:00.000000000


time: 56.4 ms


In [80]:
test1.index

MultiIndex(levels=[['dobutamine', 'dopamine', 'epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin'], [200030.0, 200033.0, 200059.0, 200065.0, 200078.0, 200095.0, 200102.0, 200116.0, 200136.0, 200140.0, 200141.0, 200143.0, 200199.0, 200231.0, 200269.0, 200282.0, 200294.0, 200342.0, 200345.0, 200349.0, 200375.0, 200391.0, 200438.0, 200441.0, 200454.0, 200457.0, 200474.0, 200475.0, 200489.0, 200504.0, 200586.0, 200636.0, 200639.0, 200758.0, 200805.0, 200863.0, 200891.0, 200897.0, 200918.0, 200938.0, 200952.0, 200983.0, 200985.0, 201005.0, 201039.0, 201046.0, 201062.0, 201074.0, 201076.0, 201094.0, 201098.0, 201116.0, 201121.0, 201186.0, 201197.0, 201201.0, 201220.0, 201286.0, 201291.0, 201299.0, 201318.0, 201329.0, 201357.0, 201358.0, 201360.0, 201361.0, 201387.0, 201404.0, 201420.0, 201427.0, 201483.0, 201555.0, 201559.0, 201593.0, 201596.0, 201608.0, 201641.0, 201652.0, 201659.0, 201667.0, 201671.0, 201695.0, 201696.0, 201718.0, 201738.0, 201748.0, 201776.0, 201791.0, 201820.

time: 502 ms


In [60]:
pd.melt(test1, col_level=0, id_vars=['charttime','delta','t_0'], var_name='label').head()
###hmm how does melt deal with multiindex

,charttime,delta,t_0,label,value
0,2139-06-02 16:31:00,0 days 16:31:00.000000000,2139-06-02,vaso_rate,0.012030
1,2139-09-23 15:07:00,0 days 15:07:00.000000000,2139-09-23,vaso_rate,0.020004
2,2139-09-25 12:00:00,2 days 12:00:00.000000000,2139-09-23,vaso_rate,0.080068
3,2139-09-25 21:59:00,2 days 21:59:00.000000000,2139-09-23,vaso_rate,0.060601
4,2139-09-23 23:12:00,0 days 23:12:00.000000000,2139-09-23,vaso_rate,0.149992


time: 27.4 ms


In [69]:
#pd.melt(test1, col_level=1, id_vars=[('vaso_type','icustay_id'),'charttime','delta','t_0'], var_name='label')
pd.melt(test1, id_vars=[('vaso_type','icustay_id')], var_name='label')


KeyError: ('vaso_type', 'icustay_id')

time: 52.5 ms


In [ ]:
# problem: i need to make a vaso_rate and vaso_amount for each vaso_type

In [44]:
vaso_dose_72['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'vasopressin', 'phenylephrine',
       'dopamine', 'dobutamine'], dtype=object)

time: 4.25 ms


In [43]:
#vaso_dose_72.spl('vaso_type')#.groupby['vaso_type']['vaso_rate']

#vaso_dose_72.pivot(index='vaso_type', columns='icustay_id', values='vaso_amount')

ValueError: Index contains duplicate entries, cannot reshape

time: 21.8 ms


In [24]:
pd.melt(vaso_dose_72, id_vars=['icustay_id','t_0','charttime','delta'], var_name='label')

,icustay_id,t_0,charttime,delta,label,value
0,200349.0,2139-06-02,2139-06-02 16:31:00,0 days 16:31:00.000000000,vaso_rate,0.0120298
1,200952.0,2139-09-23,2139-09-23 15:07:00,0 days 15:07:00.000000000,vaso_rate,0.0200044
2,200952.0,2139-09-23,2139-09-25 12:00:00,2 days 12:00:00.000000000,vaso_rate,0.0800677
3,200952.0,2139-09-23,2139-09-25 21:59:00,2 days 21:59:00.000000000,vaso_rate,0.0606006
4,200952.0,2139-09-23,2139-09-23 23:12:00,0 days 23:12:00.000000000,vaso_rate,0.149992
5,200952.0,2139-09-23,2139-09-24 21:54:00,1 days 21:54:00.000000000,vaso_rate,0.201337
6,200952.0,2139-09-23,2139-09-25 13:17:00,2 days 13:17:00.000000000,vaso_rate,0.351255
7,200952.0,2139-09-23,2139-09-25 09:24:00,2 days 09:24:00.000000000,vaso_rate,4.8
8,200952.0,2139-09-23,2139-09-23 15:06:00,0 days 15:06:00.000000000,vaso_rate,2.509
9,201005.0,2169-10-25,2169-10-25 17:25:00,0 days 17:25:00.000000000,vaso_rate,0.02


time: 74.8 ms


In [8]:
#labs_all_nosummary_72
labs_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,t_0,delta
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000
81,9,150750,220597,2149-11-13 03:20:00,ALBUMIN,3.0,2149-11-13,0 days 03:20:00.000000000
82,9,150750,220597,2149-11-13 03:20:00,ANION GAP,13.0,2149-11-13,0 days 03:20:00.000000000
83,9,150750,220597,2149-11-13 03:20:00,BICARBONATE,22.0,2149-11-13,0 days 03:20:00.000000000
84,9,150750,220597,2149-11-13 03:20:00,CHLORIDE,99.0,2149-11-13,0 days 03:20:00.000000000


time: 15.2 ms


In [ ]:
ventcategory_df_melt = pd.melt(ventcategory_df, id_vars=['icustay_id','t_0'], var_name='label')
ventcategory_df_melt.head()

In [ ]:
weightfirstday_df_melt= pd.melt(weightfirstday_df, id_vars='icustay_id', var_name='label')
weightfirstday_df_melt.head()

In [ ]:
heightfirstday_df_melt= pd.melt(heightfirstday_df, id_vars='icustay_id', var_name='label')
heightfirstday_df_melt.head()

In [ ]:
vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values
vitals_all_nosummary_nonull.head()

In [ ]:
uti_all_72_melt= pd.melt(
    uti_all_72[['hadm_id','charttime','value','valuenum','valueuom','label','fluid']],
    id_vars=['hadm_id','charttime'], var_name='label')

uti_all_72_melt_notnull= uti_all_72_melt[uti_all_72_melt['value'].notnull()] #1320954

uti_all_72_melt_notnull= uti_all_72_melt_notnull.drop_duplicates(subset=['hadm_id','label','charttime']) #drops the duplicated rows
#n=7796 rows

In [ ]:
#na values- removing all null values from value.
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72.head()

In [ ]:
#na values
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull

In [ ]:
#need to reshape the dataframe
gcs72_df_melt = pd.melt(gcs72_df, id_vars=['subject_id','hadm_id','icustay_id','t_0','ICU_admit','admit_plus_day','delta'], var_name='label')

gcs72_df_melt_notnull= gcs72_df_melt[gcs72_df_melt['value'].notnull()] #62772 ->62650
gcs72_df_melt_notnull.head()

In [ ]:
#need to reshape the dataframe
sofa_df_melt = pd.melt(sofa_df_72, id_vars=['subject_id','hadm_id','icustay_id','t_0','day','ICU_admit','approx_charttime','floor_charttime'], var_name='label')

sofa_df_melt_notnull= sofa_df_melt[sofa_df_melt['value'].notnull()] #606651 ->302459
sofa_df_melt_notnull.head()